# **Installing the necessary libararies**

In [12]:
!pip install numpy
!pip install torch
!pip install sklearn
!pip install pytorch_transformers
!pip install transformers
!pip install bertviz

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

# **Loading the Pre-trained BERT model**

In [13]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("/Users/lfoppiano/development/projects/embeddings/pre-trained-embeddings/scibert/scibert_scivocab_cased_hf")
model = AutoModel.from_pretrained("/Users/lfoppiano/development/projects/embeddings/pre-trained-embeddings/scibert/scibert_scivocab_cased_hf",output_hidden_states=True)


Some weights of the model checkpoint at /Users/lfoppiano/development/projects/embeddings/pre-trained-embeddings/scibert/scibert_scivocab_cased_hf were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Input Formatting (Tokenization)**

In [14]:
# Define a new example sentence with multiple meanings of the word "bank"
text1 = "We are studying the material La3A2Ge2 (A = Ir, Rh)."
text2 = "The critical temperature T C = 4.7 K discovered for La3Ir2Ge2 in this work is by about 1.2 K higher than that found for La3Rh2Ge2."

# Add the special tokens.
marked_text = "[CLS] " + text1 + " [SEP] " + text2 + " [SEP]"

# Split the sentence into tokens.
tokenized_text = tokenizer.tokenize(marked_text)

# Map the token strings to their vocabulary indeces.
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for idx, tup in enumerate(zip(tokenized_text, indexed_tokens)):
    print('{:<3} {:<12} {:>6,}'.format(idx, tup[0], tup[1]))

0   [CLS]           101
1   we              289
2   are             231
3   studying      9,542
4   the             111
5   material      1,804
6   la            2,326
7   ##3          30,141
8   ##a          30,108
9   ##2          30,130
10  ##ge            314
11  ##2          30,130
12  (               143
13  a               105
14  =               281
15  ir            2,519
16  ,               430
17  rh            8,651
18  )               551
19  .               211
20  [SEP]           102
21  the             111
22  critical      2,771
23  temperature   1,674
24  t               104
25  c               116
26  =               281
27  4               294
28  .               211
29  7               452
30  k               324
31  discovered    9,556
32  for             173
33  la            2,326
34  ##3          30,141
35  ##ir            220
36  ##2          30,130
37  ##ge            314
38  ##2          30,130
39  in              124
40  this            306
41  work        

# **Running BERT on the text**

In [15]:
import torch

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31116, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [16]:
# Run the text through BERT, get the output and collect all of the hidden states produced
# from all 12 layers. 
with torch.no_grad():

    outputs = model(tokens_tensor)

    # can use last hidden state as word embeddings
    last_hidden_state = outputs[0]
    word_embed_1 = last_hidden_state

    # Evaluating the model will return a different number of objects based on 
    # how it's  configured in the `from_pretrained` call earlier. In this case, 
    # becase we set `output_hidden_states = True`, the third item will be the 
    # hidden states from all layers. See the documentation for more details:
    # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
    hidden_states = outputs[2]

    # initial embeddings can be taken from 0th layer of hidden states
    word_embed_2 = hidden_states[0]

    # sum of all hidden states
    word_embed_3 = torch.stack(hidden_states).sum(0)

    # sum of second to last layer
    word_embed_4 = torch.stack(hidden_states[2:]).sum(0) 

    # sum of last four layer
    word_embed_5 = torch.stack(hidden_states[-4:]).sum(0) 

    #concat last four layers
    word_embed_6 = torch.cat([hidden_states[i] for i in [-1,-2,-3,-4]], dim=-1)




In [17]:
word_embed_5.size()

torch.Size([1, 62, 768])

In [18]:
word_embed_6.size()

torch.Size([1, 62, 3072])

In [19]:
# `hidden_states` is a Python list.
print('      Type of hidden_states: ', type(hidden_states))

# Each layer in the list is a torch tensor.
print('Tensor shape for each layer: ', hidden_states[0].size())

token_embeddings = torch.stack(hidden_states, dim=0)

# Remove batches dimension
token_embeddings = torch.squeeze(token_embeddings, dim=1)

# we swap dimension 0 (layers) and 1 (tokens)
token_embeddings = token_embeddings.permute(1,0,2)

token_embeddings.size()

      Type of hidden_states:  <class 'tuple'>
Tensor shape for each layer:  torch.Size([1, 62, 768])


torch.Size([62, 13, 768])

In [20]:
# Stores the token vectors, with shape [22 x 768]
token_vecs_sum = []

# `token_embeddings` is a [22 x 12 x 768] tensor.

# For each token in the sentence...
for token in token_embeddings:

    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
    sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))

Shape is: 62 x 768


# Analysis meaning

We are trying to check

In [21]:
for idx, tup in enumerate(zip(tokenized_text, indexed_tokens)):
    print('{:<3} {:<12} {:>6,}'.format(idx, tup[0], tup[1]))

0   [CLS]           101
1   we              289
2   are             231
3   studying      9,542
4   the             111
5   material      1,804
6   la            2,326
7   ##3          30,141
8   ##a          30,108
9   ##2          30,130
10  ##ge            314
11  ##2          30,130
12  (               143
13  a               105
14  =               281
15  ir            2,519
16  ,               430
17  rh            8,651
18  )               551
19  .               211
20  [SEP]           102
21  the             111
22  critical      2,771
23  temperature   1,674
24  t               104
25  c               116
26  =               281
27  4               294
28  .               211
29  7               452
30  k               324
31  discovered    9,556
32  for             173
33  la            2,326
34  ##3          30,141
35  ##ir            220
36  ##2          30,130
37  ##ge            314
38  ##2          30,130
39  in              124
40  this            306
41  work        

In [28]:
print('')
print("la (pos 6)", str(token_vecs_sum[6][:5]))
print("la (pos 33)", str(token_vecs_sum[33][:5]))
print("la (pos 54)", str(token_vecs_sum[54][:5]))


la (pos 6) tensor([ 1.5464,  0.0417,  2.5421,  0.1350, -3.2658])
la (pos 33) tensor([ 1.8206,  4.5411, -0.8660, -0.8465, -2.9136])
la (pos 54) tensor([ 0.5909,  4.3853,  1.2380, -2.0107, -0.4635])


In [29]:
from scipy.spatial.distance import cosine

diff = 1 - cosine(token_vecs_sum[6], token_vecs_sum[33])
print('Vector similarity for la (pos 6) and la (pos 33) meanings:  %.2f' % diff)

diff = 1 - cosine(token_vecs_sum[6], token_vecs_sum[54])
print('Vector similarity for  (pos 6) and la (pos 53) meanings:  %.2f' % diff)

diff = 1 - cosine(token_vecs_sum[33], token_vecs_sum[54])
print('Vector similarity for la (pos 33) and la (pos 53) meanings:  %.2f' % diff)

Vector similarity for la (pos 6) and la (pos 33) meanings:  0.81
Vector similarity for  (pos 6) and la (pos 53) meanings:  0.76
Vector similarity for la (pos 33) and la (pos 53) meanings:  0.87


# Display attention

In [24]:
from bertviz import model_view, head_view

attention = outputs[-1]  # Retrieve attention from model outputs

tokens = tokenizer.convert_ids_to_tokens(tokens_tensor[0])  # Convert input ids to token strings
head_view(attention, tokens)

ValueError: The attention tensor does not have the correct number of dimensions. Make sure you set output_attentions=True when initializing your model.